## **Oficina 9: Análise de sentimento com o dataset “Sentiment Labelled Sentences”**
* Dataset: [Sentiment Labelled Sentences](https://archive.ics.uci.edu/dataset/331/sentiment+labelled+sentences) 

In [1]:
# Importações
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

## Base de dados

In [2]:
# Listando a base de dados
arquivos = [
    "../Intermediario/data/amazon_cells_labelled.txt", 
    "../Intermediario/data/imdb_labelled.txt", 
    "../Intermediario/data/yelp_labelled.txt"
]
# Criando uma lista vazia para armazenar os dados
dfs = []

In [3]:
# Carregando os arquivos
for arquivo in arquivos:
    df = pd.read_csv(arquivo, delimiter='\t', header=None, names=['Texto', 'Sentimento'])
    dfs.append(df)

In [4]:
# Juntando todos os DataFrames em um só
df = pd.concat(dfs, ignore_index=True)
df.head()

,Texto,Sentimento
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [5]:
# Visualizando as informações  de dados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2748 entries, 0 to 2747
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Texto       2748 non-null   object
 1   Sentimento  2748 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 43.1+ KB


## Pré-processamento dos dados

In [6]:
# Baixando os recursos do NLTK
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\joaoi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\joaoi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
nltk.data.path.append("C:/Users/joaoi/nltk_data")

In [8]:
# Criando um função de pré-processamento
def preprocessamento(texto):
    # Deixar o texto em caixa baixa
    texto = texto.lower()
    # Remover links
    texto = re.sub(r"http\S+|www\S+|https\S+", '', texto, flags=re.MULTILINE)
    # Remover tags HTML
    texto = re.sub(r'<.*?>', '', texto)
    # Remover pontuações
    texto = re.sub(r'[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', texto)
    # Remover stopwords
    texto = ' '.join([palavra for palavra in texto.split() if palavra not in stopwords.words('english')])
    return texto

<>:10: SyntaxWarning: invalid escape sequence '\]'
<>:10: SyntaxWarning: invalid escape sequence '\]'
C:\Users\joaoi\AppData\Local\Temp\ipykernel_3136\2148229579.py:10: SyntaxWarning: invalid escape sequence '\]'
  texto = re.sub(r'[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', texto)


In [9]:
print(nltk.data.path) 

['C:\\Users\\joaoi/nltk_data', 'c:\\Labs\\venv\\nltk_data', 'c:\\Labs\\venv\\share\\nltk_data', 'c:\\Labs\\venv\\lib\\nltk_data', 'C:\\Users\\joaoi\\AppData\\Roaming\\nltk_data', 'C:\\nltk_data', 'D:\\nltk_data', 'E:\\nltk_data', 'C:/Users/joaoi/nltk_data']


In [10]:
# Aplicando a função a todos os textos
df["Texto"] = df["Texto"].apply(preprocessamento)

## Transformando o Texto em Vetores Numéricos

In [11]:
# Criando o vetorizador TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)  # Limitar a 5000 palavras mais frequentes

# Transformando os textos
X = vectorizer.fit_transform(df["Texto"])
y = df["Sentimento"]

print(X.shape) 

(2748, 5000)


## Treinando um Modelo de Machine Learning

In [12]:
# Dividindo em treino (80%) e teste (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
# Criando e treinar o modelo
modelo = MultinomialNB()
modelo.fit(X_train, y_train)

MultinomialNB()

In [14]:
# Fazendo previsões
y_pred = modelo.predict(X_test)

In [15]:
# Avaliando o modelo
print(f"Acurácia: {accuracy_score(y_test, y_pred):.2f}")
print(classification_report(y_test, y_pred))

Acurácia: 0.79
              precision    recall  f1-score   support

           0       0.85      0.74      0.79       291
           1       0.74      0.85      0.79       259

    accuracy                           0.79       550
   macro avg       0.79      0.79      0.79       550
weighted avg       0.80      0.79      0.79       550



## Conclusão
---
- Eu utilizei o modelo Naive Bayes pois ele aprende a partir dos exemplos rotulados calculando a probabilidade condicional das palavras em relação às classes. Ele assume que a presença de uma palavra em um comentário é independente da presença de qualquer outra palavra (assunção de independência).
- E para avaliar o desempenho do modelo, utilizei o conjunto de teste e calculei a acurácia, além de outras métricas como precisão, recall e f1-score. A acurácia nos dá uma ideia geral da qualidade das previsões do modelo, mas deve ser complementada com outras métricas como precisão, recall e f1-score para uma avaliação mais completa, especialmente em casos de classes desbalanceadas.